In [1]:
import pandas as pd
import numpy as np
import librosa
import os
import matplotlib.pyplot as plt
from PIL import Image

In [2]:
# DATA PREP
os.chdir('data')
os.makedirs('orca_spectrogram')
os.makedirs('noise_spectrogram')

In [3]:
orca_df = pd.read_csv("orca_data.csv")
noise_df = pd.read_csv("noise_data.csv")
noise_df.shape

(2216, 3)

In [6]:
!pwd

/home/ajm76/OrcaSeis/data


In [16]:
# Create orca audio chunks, create and save spectrogram for each
dir_path = '/home/ajm76/OrcaSeis/data'
for index, row in orca_df.iterrows():
    filename = row['wav_filename']
    filepath = 'wav/' + filename
    start_time = row['start_time_s']  # Keep in seconds
    end_time = row['end_time_s']  # Keep in seconds
    
    # Load the original wav file
    y, sr = librosa.load(filepath)

    # Convert start and end times to sample indices
    start_sample = int(start_time * sr)
    end_sample = int(end_time * sr)

    # Extract the chunk
    chunk = y[start_sample:end_sample]
        
    # Generate a Short-Time Fourier Transform (STFT) spectrogram
    D = librosa.stft(chunk)

    # Convert amplitude spectrogram to dB-scaled spectrogram
    S_db = librosa.amplitude_to_db(np.abs(D), ref=np.max)
        
    # Save the spectrogram as a NumPy array
    np.save(os.path.join(dir_path, 'orca_spectrogram', str(index)+'.npy'), S_db) 

In [17]:
# Create noise audio chunks, create and save spectrogram for each
dir_path = '/home/ajm76/OrcaSeis/data'
for index, row in noise_df.iterrows():
    filename = row['wav_filename']
    filepath = 'wav/' + filename
    start_time = row['start_time_s']  # Keep in seconds
    end_time = row['end_time_s']  # Keep in seconds
    
    # Load the original wav file
    y, sr = librosa.load(filepath)

    # Convert start and end times to sample indices
    start_sample = int(start_time * sr)
    end_sample = int(end_time * sr)

    # Extract the chunk
    chunk = y[start_sample:end_sample]
        
    # Generate a Short-Time Fourier Transform (STFT) spectrogram
    D = librosa.stft(chunk)

    # Convert amplitude spectrogram to dB-scaled spectrogram
    S_db = librosa.amplitude_to_db(np.abs(D), ref=np.max)
        
    # Save the spectrogram as a NumPy array
    np.save(os.path.join(dir_path, 'noise_spectrogram', str(index)+'.npy'), S_db) 

/home/jupyter_share/miniconda3/envs/seismo2/lib/python3.10/site-packages/librosa/core/spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=0
  warnings.warn(
/home/jupyter_share/miniconda3/envs/seismo2/lib/python3.10/site-packages/librosa/core/spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=512
  warnings.warn(
/home/jupyter_share/miniconda3/envs/seismo2/lib/python3.10/site-packages/librosa/core/spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=22
  warnings.warn(
/home/jupyter_share/miniconda3/envs/seismo2/lib/python3.10/site-packages/librosa/core/spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=11
  warnings.warn(
/home/jupyter_share/miniconda3/envs/seismo2/lib/python3.10/site-packages/librosa/core/spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=19
  warnings.warn(
/home/jupyter_share/miniconda3/envs/seismo2/lib/python3.10/site-p

In [18]:
# ML
input_dir = "/home/ajm76/OrcaSeis/data"
categories = ['noise', 'orca_call']

In [21]:
data = []
labels = []
for category_idx, category in enumerate(categories):
    cat_path = ""
    if category == 'noise':
        cat_path = 'noise_spectrogram' # actual code
#         cat_path = 'noise_test' # just for testing
    else:
        cat_path = 'orca_spectrogram' # actual code
#         cat_path = 'orca_test'
    for file in os.listdir(os.path.join(input_dir, cat_path)):
        spectrogram = np.load(os.path.join(input_dir, cat_path, file))
        data.append(spectrogram)
        labels.append(category_idx)

In [ ]:
# # Convert all elements to NumPy arrays if they aren't already
# data = [np.array(spectrogram) for spectrogram in data]

# # Determine the maximum shape for each dimension
# max_shape = tuple(max(dim) for di/m in zip(*[spectrogram.shape for spectrogram in data]))

# # Initialize a list to store the padded spectrograms
# standardized_data = []

# for spectrogram in data:
#     # Pad spectrograms to match the maximum shape
#     pad_widths = [(0, max_size - size) for size, max_size in zip(spectrogram.shape, max_shape)]
#     padded_spectrogram = np.pad(spectrogram, pad_widths, mode='constant', constant_values=0)
#     standardized_data.append(padded_spectrogram)

# # Convert to a single NumPy array
# data = np.array(standardized_data)

# print(data.shape)